## Programação Linear Inteira
***

##### Problemas de programação linear inteira (PLI) são problemas de programação linear (PL) em que todas ou algumas de suas variáveis assumem valores discretos, isto é, valores pertencentes ao conjunto dos números inteiros.

##### Quanto todas as variáveis são números inteiros, teremos um problema de progrmação linear inteira pura (PLIP). Caso contrário, programação linear inteira mista (PLIM).

##### Um caso especial de variável inteira é a variável binária, que assume somente valores iguais a zero ou um. Quando todas as variáveis de um modelo são binárias, diz-se que o modelo é de programação inteira binária.

##### Em geral, os problemas apresentados estão sujeitos à restrições. Essas restrições impõem condições às variáveis do problema.

[Ler mais.](https://repositorio-aberto.up.pt/bitstream/10216/74369/2/40539.pdf)

### Problema da Mochila (Knapsack Problem)
***

##### Um alpinista dispõe de uma mochila e de diversos objetos que podem ser carregados na mochila; para cada objeto é conhecido o seu peso (wi) e o benefício que dele se tira (pi). Conhecida a capacidade da mochila em termos de peso máximo (c), quais os objetos, dos n existentes, que devem ser escolhidos para colocar na mochila?

##### Observação 1: Queremos maximizar a quantidade de itens dentro da mochila. No entanto, levaremos em consideração não apenas a quantidade de itens, mas também o benefício que teremos ao levá-los, ou seja, o quanto o item escolhido é importante para a viagem. Este é nosso objetivo.

##### Observação 2: É necessário respeitar a capacidade da mochila. Assim sendo, queremos a maior quantidade de itens importantes e que, com seus respectivos pesos somados, não ultrapasse o valor c. Esta é uma restrição.

##### Em termos matemáticos:

<center><img src = 'assets/mochila.png'></center>

##### A variável x é uma variável binária que assume seu valor da seguinte forma:

> Caso o objeto seja escolhido, seu valor será igual a 1.

> Caso o objeto não seja escolhido, seu valor será igual a 0.

***
### Resolução:

##### Para resolvermos o problema, usaremos uma biblioteca chamada [MIP](https://docs.python-mip.com/en/latest/examples.html).

In [26]:
from mip import *
import pandas as pd

##### Criação de dados sintéticos:

In [9]:
# Benefícios dos itens (p: profit)
p = [10, 13, 18, 31, 7, 15]

# Pesos dos objetos (w: weight)
w = [11, 15, 20, 35, 10, 33]

# Capacidade da mochila
c = 47

# Quantidade de itens
I = range(len(w))

##### Definição do modelo e das restrições:

In [10]:
# Definição do modelo
m = Model('knapsack', solver_name=CBC)

# Definição da variável binária x:
x = [m.add_var(var_type=BINARY) for i in I]

# Objetivo: Maximizar a quantidade de itens x seu benefício:
m.objective = maximize(xsum(p[i] * x[i] for i in I))

# Restrição: Atender à capacidade da mochila
m += xsum(w[i] * x[i] for i in I) <= c

# Otimizando o modelo:
m.optimize()

Cgl0004I processed model has 1 rows, 6 columns (6 integer (6 of which binary)) and 6 elements
Coin3009W Conflict graph built in 0.002 seconds, density: 14.103%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I Nauty did not find any useful orbits in time 0
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.457143
Cbc0038I Solution found of -28
Cbc0038I Before mini branch and bound, 5 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1 rows 6 columns, reduced to 0 rows 0 columns
Cbc0038I Mini branch and bound did not improve solution (0.02 seconds)
Cbc0038I Round again with cutoff of -30.3171
Cbc0038I Reduced cost fixing fixed 1 variables on major pass 2
Cbc0038I Pass   1: suminf.    0.07474 (1) obj. -30.3171 iterations 1
Cbc0038I Pass   2: suminf.    0.45714 (1) obj. -42.1714 iterations 1
Cbc0038I Pass   3: suminf.    0.07474 (1) obj. -30.3171 iterations 1
Cbc0038I Pass   4: suminf.    0.45714 (1) obj. -42.1714 iterations 1
Cbc0038I Pass   5:

<OptimizationStatus.OPTIMAL: 0>

##### Visualização dos itens selecionados:

In [11]:
# Os itens que possuírem x = 0 (não selecionados) não serão mostrados
selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [0, 3]


##### Assim sendo, os itens selecionados foram: item 0 e item 3.

In [12]:
m.objective_value

41.0

##### O valor ótimo de benefício (máximo benefício possível, de acordo com as restrições de capacidade da mochila) soma 41 pontos.

In [13]:
print(m.vars[3].x)

1.0


***
### Problema das ações
***

##### Um investidor da bolsa planeja investir em ações nos próximos 3 anos. As 5 ações disponíveis se encontram na tabela abaixo, bem como suas respectivas previsões de retorno. O investimento necessário a cada ação ao longo dos 3 anos e o limite de gasto do do investidor também estão na tabela.

<center><img src = 'assets/1_acoes.png'></center>

##### Identificando as variáveis

* Número de ações: I
* Aporte necessário para cada ação em cada ano: a[t][i]
* Limite de gastos do investidor para cada ano: limite[t]
* Retorno de cada ação: r[i]

##### Variável de decisão:

* x[t][i]: Variável binária.

##### Objetivo:

> Maximizar o retorno ANUAL dos investimentos

##### Restrição:

> A soma dos investimentos anuais tem que ser menor ou igual ao orçamento do investidor para aquele ano.

<center><img src = 'assets/2_acoes.png'></center>

##### Variáveis

In [14]:
# Retornos ANUAIS dos projetos:

r = [90, 120, 100, 80, 130]

# Limite de investimento a cada ano
limite = [45, 60, 50]

# Aportes necessários em cada ano, para cada ação:
a = [
     [10, 15, 12, 9, 13],
     [20, 15, 25, 15, 10],
     [15, 20, 20, 15, 10]
    ]

# Quantidade de anos:
T = range(len(limite))

# Número de ações:
I = range(len(r))

##### Instanciando o modelo

In [15]:
m = Model(sense=MAXIMIZE, solver_name=CBC)

##### Variável binária

In [17]:
x = [[m.add_var(var_type=BINARY) for i in I] for t in T]

In [19]:
# Note que temos uma variável x para cada ação (são 5), para cada ano (são 3)

x

[[<mip.entities.Var at 0x7fd134719c40>,

##### Objetivo: Maximizar o retorno ANUAL dos investimentos

In [20]:
m.objective = maximize(xsum(r[i]*x[t][i] for i in I for t in T))

##### Restrições: A cada ano, o total dos investimentos precisa obedecer ao orçamento do investidor

In [21]:
for t in range(3):
    m += xsum(a[t][i] * x[t][i] for i in I) <= limite[t]

In [22]:
# O OPTIMAL é retornado quando o processamento trmina e a solução ótima é encontrada

status = m.optimize() 
if status == OptimizationStatus.OPTIMAL:  
 print(f'optimal solution cost {m.objective_value} found') 

Cgl0004I processed model has 1 rows, 6 columns (6 integer (6 of which binary)) and 6 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 14.103%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I Nauty did not find any useful orbits in time 2.8e-05
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.457143
Cbc0038I Solution found of -28
Cbc0038I Before mini branch and bound, 5 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1 rows 6 columns, reduced to 0 rows 0 columns
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I Round again with cutoff of -30.3171
Cbc0038I Reduced cost fixing fixed 1 variables on major pass 2
Cbc0038I Pass   1: suminf.    0.07474 (1) obj. -30.3171 iterations 1
Cbc0038I Pass   2: suminf.    0.45714 (1) obj. -42.1714 iterations 1
Cbc0038I Pass   3: suminf.    0.07474 (1) obj. -30.3171 iterations 1
Cbc0038I Pass   4: suminf.    0.45714 (1) obj. -42.1714 iterations 1
Cbc0038I Pas

##### Resultados:

In [23]:
results = {}
for i in range(3):  
    results[i+1] = [m.vars[j].x for j in range(5*i,5*i+5)]

In [24]:
results

{1: [1.0, 0.0, 1.0, 1.0, 1.0],
 2: [1.0, 1.0, 0.0, 1.0, 1.0],
 3: [0.0, 1.0, 1.0, 0.0, 1.0]}

##### Alocando em um dataframe o resultado

In [27]:
result = pd.DataFrame(data = results, index = ['A','B','C','D','E'])
result

,1,2,3
A,1.0,1.0,0.0
B,0.0,1.0,1.0
C,1.0,0.0,1.0
D,1.0,1.0,0.0
E,1.0,1.0,1.0


##### Conclusão:

* No primeiro ano, devem ser compradas as ações A, C, D e E
* No segundo, A, B, D e E
* no terceiro, B, C e E.

##### Nestes termos, teremos o retorno máximo possível.

***
### Problema da Seleção de Projetos
***

##### Um empresário está fazendo um planejamento para os próximos 3 anos. Ele tem disponíveis R$25 milhões para investir anualmente e 5 projetos estão sob consideração. A tabela abaixo apresenta os retornos esperados para cada projeto ao final do terceiro ano e seus respectivos desembolsos anuais.

<center><img src = 'assets/projetos.png'></center>

##### Quais projetos devem ser escolhidos pelo empresário?

***
### Resolução:

##### Nosso objetivo é maximizar o retorno dos investimentos, isto é, a soma dos retornos dos projetos escolhidos tem que ser máxima.

In [28]:
# Retorno de cada um dos projetos:

r = [20, 40, 20, 15, 30]

# Investimento disponível para cada um dos anos
limite = [25, 25, 25]

# Aportes necessários para cada projeto em cada um dos 3 anos:
a = [
      [5, 4, 3, 7, 8],  #ano 1
      [1, 7, 9, 4, 6],  #ano 2
      [8, 10, 2, 1, 10] #ano 3
    ]

# Projetos disponíveis:
P = range(len(r))

# Quantidade de anos:
T = range(len(limite))

In [29]:
from mip.model import *

m_cbc = Model(sense = MAXIMIZE, solver_name = CBC)

In [30]:
# Definição da variável binária x:
x = [m_cbc.add_var(var_type=BINARY) for projeto in P]

In [32]:
# Objetivo: Maximizar o retorno obtido dos investimentos:
m_cbc.objective = maximize(xsum(r[projeto] * x[projeto] for projeto in P))

# Restrições: Atender ao limite de investimento em cada um dos anos

for ano in range(3):  
    m_cbc += xsum(a[ano][projeto] * x[projeto] for projeto in P) <= limite[ano]

# Otimizando o modelo:
m_cbc.optimize()

 42: suminf.    0.68095 (2) obj. -1181.9 iterations 2
Cbc0038I Pass  43: suminf.    0.82738 (3) obj. -1181.9 iterations 2
Cbc0038I Pass  44: suminf.    0.60476 (2) obj. -1181.9 iterations 3
Cbc0038I Pass  45: suminf.    0.60476 (2) obj. -1181.9 iterations 0
Cbc0038I Pass  46: suminf.    0.60714 (2) obj. -1182.14 iterations 2
Cbc0038I Pass  47: suminf.    0.44246 (2) obj. -1181.9 iterations 4
Cbc0038I Pass  48: suminf.    0.33333 (2) obj. -1181.9 iterations 2
Cbc0038I Pass  49: suminf.    0.44246 (2) obj. -1181.9 iterations 2
Cbc0038I Pass  50: suminf.    0.33333 (2) obj. -1181.9 iterations 2
Cbc0038I Pass  51: suminf.    0.44246 (2) obj. -1181.9 iterations 2
Cbc0038I Pass  52: suminf.    0.33333 (2) obj. -1181.9 iterations 2
Cbc0038I Pass  53: suminf.    0.53968 (2) obj. -1181.9 iterations 3
Cbc0038I Pass  54: suminf.    0.53968 (2) obj. -1181.9 iterations 0
Cbc0038I Pass  55: suminf.    0.68095 (2) obj. -1181.9 iterations 3
Cbc0038I Pass  56: suminf.    0.59523 (3) obj. -1181.9 iterat

<OptimizationStatus.OPTIMAL: 0>

In [44]:
status = m_cbc.optimize() 
if status == OptimizationStatus.OPTIMAL:  
 print('optimal solution cost {} found'.format(m_cbc.objective_value)) 

Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 3 rows, 5 columns (5 integer (5 of which binary)) and 15 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 9.091%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I MIPStart provided solution with cost 95
Cbc0012I Integer solution of 95 found by Reduced search after 0 iterations and 0 nodes (0.00 seconds)
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0045I Solution of -95 already found by heuristic
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Clp0000I Optimal - objective value 108.68421

Starting MIP optimization
optimal solution cost 95.0 found


In [43]:
for projeto in P:
    print(m_cbc.vars[projeto].x)

1.0
1.0
1.0
1.0
0.0


##### Resultados:

* O valor ótimo encontrado, ou seja, o maior retorno possível APÓS os 3 anos de investimento será de R$95 milhões.
* Serão escolhidos os projetos 1, 2, 3 e 4. O projeto 5 não será escolhido.

***
##### Incluindo mais restrições:

> a) Não se pode selecionar mais do que 3 projetos

In [46]:
m_cbc_a = Model(sense = MAXIMIZE, solver_name = CBC)

# Definição da variável binária x:
x = [m_cbc_a.add_var(var_type=BINARY) for projeto in P]

# Objetivo: Maximizar o retorno obtido dos investimentos:
m_cbc_a.objective = maximize(xsum(r[projeto] * x[projeto] for projeto in P))

# Restrições: Atender ao limite de investimento em cada um dos anos

for ano in range(3):  
    m_cbc_a += xsum(a[ano][projeto] * x[projeto] for projeto in P) <= limite[ano]
    m_cbc_a += xsum(x[projeto] for projeto in P) <= 3

# Otimizando o modelo:
m_cbc_a.optimize()

Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 3 rows, 5 columns (5 integer (5 of which binary)) and 15 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 9.091%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I MIPStart provided solution with cost 95
Cbc0012I Integer solution of 95 found by Reduced search after 0 iterations and 0 nodes (0.00 seconds)
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0045I Solution of -95 already found by heuristic
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 4 (-2) rows, 5 (0) columns and 20 (-

<OptimizationStatus.OPTIMAL: 0>

In [47]:
status = m_cbc_a.optimize() 
if status == OptimizationStatus.OPTIMAL:  
 print('optimal solution cost {} found'.format(m_cbc_a.objective_value)) 

Cgl0004I processed model has 4 rows, 5 columns (5 integer (5 of which binary)) and 20 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 9.091%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I Nauty did not find any useful orbits in time 5.3e-05
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -90
Cbc0038I Before mini branch and bound, 5 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of -90 - took 0.00 seconds
Cbc0012I Integer solution of -90 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -90, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Clp0000I Optimal - objective value 90

Start

In [48]:
for projeto in P:
    print(m_cbc_a.vars[projeto].x)

0.0
1.0
1.0
0.0
1.0


##### Neste caso, o retorno ótimo será de R$90 milhões e os projetos escolhidos serão: 2, 3 e 5.

> b) O projeto 3 deve ser selecionado se o projeto 2 for selecionado.

In [49]:
m_cbc_b = Model(sense = MAXIMIZE, solver_name = CBC)

# Definição da variável binária x:
x = [m_cbc_b.add_var(var_type=BINARY) for projeto in P]

# Objetivo: Maximizar o retorno obtido dos investimentos:
m_cbc_b.objective = maximize(xsum(r[projeto] * x[projeto] for projeto in P))

# Restrições: Atender ao limite de investimento em cada um dos anos

for ano in range(3):  
    m_cbc_b += xsum(a[ano][projeto] * x[projeto] for projeto in P) <= limite[ano]
    m_cbc_b += x[1] <= x[2]

# Otimizando o modelo:
m_cbc_b.optimize()

Cgl0004I processed model has 4 rows, 5 columns (5 integer (5 of which binary)) and 20 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 9.091%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I MIPStart provided solution with cost 90
Cbc0012I Integer solution of 90 found by Reduced search after 0 iterations and 0 nodes (0.00 seconds)
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0045I Solution of -90 already found by heuristic
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 4 (-2) rows, 5 (0) columns and 17 (-4) elements
Clp1000I sum of infeasibilities 1.1422e-11 - average 2.85549e-12, 1 fixed columns
Coin0506I Presolve 4 (0) rows, 4 (-1) columns and 14 (-3) elements
Clp0006I 0  Obj 108.68414 Dual inf 11500 (4)
Clp0029I End of values pass after 4 iterations
Clp0000I Optimal - objective value 108.68421
Cl

<OptimizationStatus.OPTIMAL: 0>

In [50]:
status = m_cbc_b.optimize() 
if status == OptimizationStatus.OPTIMAL:  
 print('optimal solution cost {} found'.format(m_cbc_b.objective_value)) 

r 2 (Knapsack) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 3 (Clique) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 4 (OddWheel) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 5 (MixedIntegerRounding2) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 6 (FlowCover) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0001I Search completed - best objective -95, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 1 variables fixed on reduced cost
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Clp0000I Optimal - objective value 108.68421

Starting MIP optimization
optimal solution cost 

In [51]:
for projeto in P:
    print(m_cbc_b.vars[projeto].x)

1.0
1.0
1.0
1.0
0.0
